In [42]:
from glob import glob
import numpy as np
import random
import function as f
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import csv

In [43]:
import monai
from monai.transforms import (
    Compose,
    LoadImage,
    Activations,
    AsDiscrete,
    SqueezeDim,
    AddChannel,
    ScaleIntensity,
    RandRotate,
    RandFlip,
    RandZoom,
    EnsureType,
    Resize
    #AddChannel,
    #RandRotate90,
    #RandSpatialCrop,
    #ScaleIntensity,
)
from monai.data import ArrayDataset, decollate_batch
from monai.metrics import DiceMetric, ROCAUCMetric
from monai.inferers import sliding_window_inference
from monai.networks.nets import DenseNet264, DenseNet121
from monai.optimizers import generate_param_groups

In [44]:
from monai.transforms import (
    Activations,
    AsDiscrete,
    Compose,
    ScaleIntensity,
)

dicom_imtrans = Compose(ScaleIntensity())
post_trans = Compose([Activations(sigmoid=True), AsDiscrete(threshold_values=True)])

In [45]:
def pad_to_square(a, LR, pad_value=0):
    m = a.reshape((a.shape[0], -1))
    padded = pad_value * np.ones(2 * [max(m.shape)], dtype=m.dtype)
    if  LR != 0 :
        padded[0:m.shape[0], 0:m.shape[1]] = m # Left
    else :
        padded[:,-m.shape[1]:] = m # Right
    return padded

def get_resize(image, dim, LR):
    image_square = pad_to_square(image, LR)
    image_resize = cv2.resize(image_square, dim, interpolation = cv2.INTER_AREA)
    return image_resize

In [46]:
import getfinal as f

def get_predict(model, dicom, method="unet",roi_size = (96, 96), sw_batch_size = 4):
    image = f.get_final5(dicom)
    LR = image[0,0]
    dim_original = max(image.shape)
    image = get_resize(image/image.max(),dim=(512,512),LR=LR)
    image = torch.Tensor(dicom_imtrans(image)[None,None,:])
    model.eval()
    with torch.no_grad():
        #roi_size = (96, 96)
        #sw_batch_size = 4
        #val_outputs = sliding_window_inference(image, roi_size, sw_batch_size, model)
        val_outputs = model(image)
        if method == "deeplab":
            val_outputs = val_outputs['out']
        val_outputs = post_trans(val_outputs)
    image = image.to("cpu").numpy()[0][0]
    segment = one_hot_reverse(val_outputs.to("cpu").numpy().astype(int)[0])    
    return image, segment, dim_original
def plot_predict(model, dicom, method="unet", roi_size = (96, 96), sw_batch_size = 4):
    image, segment, dim_original = get_predict(model, dicom, method=method, roi_size = roi_size, sw_batch_size = sw_batch_size)
    plot_seg(image, segment)
    
def cal_predict(model, dicom, method="unet", roi_size = (96, 96), sw_batch_size = 4):    
    image, segment, dim_original = get_predict(model, dicom, method=method, roi_size = roi_size, sw_batch_size = sw_batch_size)
    density_pixel = [np.sum(image!=0), np.sum(segment>=1), np.sum(segment>=2), np.sum(segment>=3)]

    density_area = [i * ((dim_original*dim_original)/(512*512)) for i in density_pixel]    

    scaling = f.get_info(dicom, 7)
    density_area = [i * (float(scaling[0]) * float(scaling[1])) / 100 for i in density_area]

    density_percent = (density_pixel/density_pixel[0] * 100).tolist()
    density_pred = {"pixel":density_pixel, "area":density_area, "percent":density_percent}

    return density_pred

In [63]:
###########################################################################################
file = list_cal[0]
pred = cal_predict(model_unet, file, method="unet")
print(pred['pixel'], pred['area'], pred['percent'])
##############################################################################

[69009, 232260, 221222, 174262] [110.8593867359471, 373.11366869960546, 355.3816930038067, 279.94288355692174] [100.0, 336.56479589618743, 320.5697807532351, 252.5206857076613]


In [47]:
def one_hot_reverse(segs):
    segment = segs[1].copy()
    segment[segs[0]==1] = 0    
    segment[segs[2]==1] = 2    
    segment[segs[3]==1] = 3
    return segment

In [48]:
def plot_seg(im, segment,cmap="jet"):
    plt.imshow(im, cmap=plt.cm.gray)
    plt.imshow(segment, cmap=cmap, alpha=0.5) #, interpolation='bilinear')
    plt.show()

In [49]:
def get_risk(model, dicom):
    image = f.get_final5(dicom)
    LR = image[0,0]
    image = get_resize(image/image.max(),dim=(512,512),LR=LR)
    image = torch.Tensor(dicom_imtrans(image)[None,None,:])
    model.eval()
    with torch.no_grad():
        result = model(image)
        prop_softmax = torch.nn.Softmax(dim=1)(result).to("cpu").numpy()[0].tolist()
        prop_sigmoid = torch.sigmoid(result).to("cpu").numpy()[0].tolist()
        #prediction = int(result.argmax(dim=1).to("cpu").numpy())
    return prop_softmax, prop_sigmoid

In [50]:
BATCH_SIZE = 1
# for name, _ in net.named_modules(): print(name)
#occ_sens = monai.visualize.OcclusionSensitivity(nn_module=target_model, mask_size=10, n_batch=BATCH_SIZE, stride=4)

def get_gradcam(model, dicom, mask_size, stride):    
    target_model = model.module
    target_layer = "class_layers.relu"
    gradcam = monai.visualize.GradCAM(nn_module=target_model, target_layers=target_layer)
    occ_sens = monai.visualize.OcclusionSensitivity(nn_module=target_model, mask_size=mask_size, n_batch=BATCH_SIZE, stride=stride)
    
    image = f.get_final5(dicom)
    LR = image[0,0]
    image = get_resize(image/image.max(),dim=(512,512),LR=LR)
    image = torch.Tensor(dicom_imtrans(image)[None,None,:])
    image = image.to(device)
    model.eval()
    with torch.no_grad():
        result = model(image)
        pred_label = result.argmax(1).item()
        prop_softmax = torch.nn.Softmax(dim=1)(result).to("cpu").numpy()[0].tolist()
        prop_sigmoid = torch.sigmoid(result).to("cpu").numpy()[0].tolist()
    img = image.detach().cpu().numpy()[0][0]
    cam_result = gradcam(x=image, class_idx=None).cpu().detach().numpy()[0][0]
    occ_map, occ_most_prob = occ_sens(x=image)
    occ_map = occ_map.cpu().detach().numpy()[0][0]
    occ_most_prob = occ_most_prob.cpu().detach().numpy()[0][0]
    occ_map1 = occ_map[...,0]
    occ_map2 = occ_map[...,1]
    return prop_sigmoid, img, cam_result, occ_map1, occ_map2, occ_most_prob

### dataset

In [51]:
list_dicom = glob("dicom/*/*")
df_image = pd.read_csv("csv/train.csv")

In [52]:
df_cal = df_image.loc[(df_image.dataset_density=="yes")&(df_image.dataset=="cal"),["CENTER","file",'GROUP',"AGE","BMI"]]
list_cal = ('dicom/'+ df_cal.CENTER + '/' + df_cal.file).tolist()

In [53]:
df_test = df_image.loc[(df_image.dataset_density=="yes")&(df_image.dataset=="test"),["CENTER","file",'GROUP',"AGE","BMI"]]
list_test = ('dicom/'+ df_test.CENTER + '/' + df_test.file).tolist()

In [54]:
import re
df_test_case = df_test[df_test.GROUP=="case"].copy()
df_test_case["file"] = [re.sub("LCC","RCC",i) if "LCC" in i else re.sub("RCC","LCC",i) for i in df_test_case.file]
list_test_case = ('dicom/'+ df_test_case.CENTER + '/' + df_test_case.file).tolist()

In [55]:
df_benign = df_image[df_image.GROUP=="benign"]
list_benign = ('dicom/'+ df_benign.CENTER + '/' + df_benign.file).tolist()
df_ilsan = df_image[df_image.CENTER=="ilsan"]
list_ilsan = ('dicom/'+ df_ilsan.CENTER + '/' + df_ilsan.file).tolist()

In [56]:
df_samsung = df_image[df_image.CENTER=="samsung"].copy()
df_samsung.loc[:,['ID']] = [int(i) for i in df_samsung.ID]
df_samsung = df_samsung.merge(pd.read_csv("csv/density_samsung.csv"), on=["ID"])
df_samsung = df_samsung[["file","darea_cu","darea_ac","darea_cc","pda_cu","pda_ac","pda_cc"]]

<ipython-input-56-0d19d5cc1840>:2: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_samsung.loc[:,['ID']] = [int(i) for i in df_samsung.ID]


In [57]:
list_samsung = ('dicom/samsung/'+ df_samsung.file).tolist()

### Model

In [58]:
import torch
#model_deeplab = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True)
#model_deeplab.backbone.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
#model_deeplab.classifier[4] = torch.nn.Conv2d(256, 4, kernel_size=(1, 1), stride=(1, 1))
#model_deeplab.aux_classifier[4] = torch.nn.Conv2d(256, 4, kernel_size=(1, 1), stride=(1, 1))

import monai
model_unet = monai.networks.nets.UNet(
    dimensions=2,
    in_channels=1,
    out_channels=4,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs")
    model_unet = torch.nn.DataParallel(model_unet, device_ids = [0, 1, 2])
    #model_deeplab = torch.nn.DataParallel(model_deeplab, device_ids = [0, 1, 2])

model_unet = model_unet.to(device)
#model_deeplab = model_deeplab.to(device)

model_unet.load_state_dict(torch.load("model/density-unet.pth"))
#model_deeplab.load_state_dict(torch.load("model/density-deeplab.pth"))
#model_deeplab.load_state_dict(torch.load("model/density-deeplab_0.5-1.0.pth"))

Using 3 GPUs


In [59]:
model_image = DenseNet121(spatial_dims=2, in_channels=1, out_channels=2)
device = torch.device("cuda:0")
if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs")
    model_image = torch.nn.DataParallel(model_image, device_ids = [0, 1, 2])
model_image = model_image.to(device)
model_image.load_state_dict(torch.load("model/image2.pth"))

Using 3 GPUs


### Predict

### predict_density.csv / predict_image.csv

In [60]:
name1 = 'csv/predict_density.csv'

with open(name1, 'w') as result: 
    write = csv.writer(result)
    write.writerow(['file','dataset','pixel_ta_dl','pixel_cu_dl','pixel_ac_dl','pixel_cc_dl',
                               'area_ta_dl','area_cu_dl','area_ac_dl','area_cc_dl',
                               'percent_ta_dl','percent_cu_dl','percent_ac_dl','percent_cc_dl'])

In [61]:
name2 = 'csv/predict_image.csv'

with open(name2, 'w') as result: 
    write = csv.writer(result)
    write.writerow(['file','dataset','softmax_prob','sigmoid_prob','softmax_label','sigmoid_label'])

In [80]:
for i in tqdm(range(len(df_cal)),miniters=50, mininterval=5):
    file = list_cal[i]
    pred = cal_predict(model_unet, file, method="unet")
    
    with open(name1, 'a') as result: 
        write = csv.writer(result)
        write.writerow([file,'cal']+pred['pixel'] + pred['area'] + pred['percent'])
        
    prob = get_risk(model_image, file)
    with open(name2, 'a') as save: 
        write = csv.writer(save)
        write.writerow([file, 'cal' ,prob[0][1], prob[1][1], np.argmax(np.array(prob[0])), np.argmax(np.array(prob[1]))])        

  0%|          | 3/809 [00:02<11:27,  1.17it/s]


KeyboardInterrupt: 

In [22]:
## 오류해결
file = list_cal[100]
pred = cal_predict(model_unet, file, method="unet")
print(pred['pixel'], pred['area'], pred['percent']) ##ㅠㅠ
#print(prob[0][1], prob[1][1], np.argmax(np.array(prob[0])), np.argmax(np.array(prob[1])))

[67593, 247994, 242379, 236351] [111.39060442550993, 408.68435420679526, 399.4310551395954, 389.4971483226621] [100.0, 366.8930214667199, 358.5859482490791, 349.66786501560813]


In [26]:
for i in tqdm(range(len(df_test)),miniters=50, mininterval=5):
    file = list_test[i]
    pred = cal_predict(model_unet, file, method="unet")
    
    with open(name1, 'a') as result: 
        write = csv.writer(result)
        write.writerow([file,'test']+pred['pixel'] + pred['area'] + pred['percent'])
        
    prob = get_risk(model_image, file)
    with open(name2, 'a') as save: 
        write = csv.writer(save)
        write.writerow([file,'test', prob[0][1], prob[1][1], np.argmax(np.array(prob[0])), np.argmax(np.array(prob[1]))])        

 18%|█▊        | 143/808 [01:28<06:37,  1.67it/s]/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
 18%|█▊        | 148/808 [01:34<07:53,  1.39it/s]/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarni

/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format h

/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
 22%|██▏       | 178/808 [02:19<15:22,  1.46s/it]/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 20

In [ ]:
name1 = 'csv/predict_density.csv'
name2 = 'csv/predict_image.csv'
for i in tqdm(range(len(df_ilsan)),miniters=50, mininterval=5):
    file = list_ilsan[i]
    pred = cal_predict(model_unet, file, method="unet")
    
    with open(name1, 'a') as result: 
        write = csv.writer(result)
        write.writerow([file,'ilsan']+pred['pixel'] + pred['area'] + pred['percent'])
        
    prob = get_risk(model_image, file)
    with open(name2, 'a') as save: 
        write = csv.writer(save)
        write.writerow([file,'ilsan', prob[0][1], prob[1][1], np.argmax(np.array(prob[0])), np.argmax(np.array(prob[1]))])        

In [ ]:
file = list_ilsan[0]
pred = cal_predict(model_unet, file, method="unet")
pred

In [ ]:
name1 = 'csv/predict_density.csv'
name2 = 'csv/predict_image.csv'
for i in tqdm(range(len(df_test_case)),miniters=50, mininterval=5):
    file = list_test_case[i]
    try:
        pred = cal_predict(model_unet, file, method="unet")
    
        with open(name1, 'a') as result: 
            write = csv.writer(result)
            write.writerow([file,'test_case']+pred['pixel'] + pred['area'] + pred['percent'])

        prob = get_risk(model_image, file)
        with open(name2, 'a') as save: 
            write = csv.writer(save)
            write.writerow([file, 'test_case', prob[0][1], prob[1][1], np.argmax(np.array(prob[0])), np.argmax(np.array(prob[1]))])        
    except:
        pass

In [ ]:
name1 = 'csv/predict_density.csv'
name2 = 'csv/predict_image.csv'
for i in tqdm(range(len(df_benign)),miniters=50, mininterval=5):
    file = list_benign[i]
    pred = cal_predict(model_unet, file, method="unet")
    
    with open(name1, 'a') as result: 
        write = csv.writer(result)
        write.writerow([file,'benign']+pred['pixel'] + pred['area'] + pred['percent'])
        
    prob = get_risk(model_image, file)
    with open(name2, 'a') as save: 
        write = csv.writer(save)
        write.writerow([file,'benign', prob[0][1], prob[1][1], np.argmax(np.array(prob[0])), np.argmax(np.array(prob[1]))])        

### test_density_test.csv

In [27]:
name = 'test_density_test.csv'
with open(name, 'w') as result: 
    write = csv.writer(result)
    write.writerow(['file','pixel_ta_dl','pixel_cu_dl','pixel_ac_dl','pixel_cc_dl',
                               'area_ta_dl','area_cu_dl','area_ac_dl','area_cc_dl',
                               'percent_ta_dl','percent_cu_dl','percent_ac_dl','percent_cc_dl'])

for i in tqdm(range(len(df_test)),miniters=50, mininterval=5):
    file = list_test[i]
    try:
        pred = cal_predict(model_unet, file,method="unet")
        with open(name, 'a') as result: 
            write = csv.writer(result)
            write.writerow([file]+pred['pixel'] + pred['area'] + pred['percent'])
    except:
        pass

 18%|█▊        | 142/808 [00:39<03:05,  3.59it/s]/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 20

/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format h

/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format h

/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format h

/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
 33%|███▎      | 268/808 [02:17<07:24,  1.22it/s]/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 20

...

/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
 85%|████████▌ | 690/808 [08:14<01:42,  1.15it/s]/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 20

/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
 89%|████████▉ | 720/808 [08:41<01:19,  1.11it/s]/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 20

/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
 93%|█████████▎| 749/808 [09:08<00:54,  1.09it/s]/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 20

/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
 96%|█████████▋| 778/808 [09:36<00:28,  1.04it/s]/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 20

100%|█████████▉| 807/808 [10:05<00:00,  1.03it/s]/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
100%|██████████| 808/808 [10:06<00:00,  1.33it/s]


In [30]:
dict_density = {}
for i in tqdm(range(len(df_cal)),miniters=50, mininterval=5):
    file = list_cal[i]
    pred = cal_predict(model_unet, file, method="unet")
    dict_density[file] = pred['pixel'] + pred['area'] + pred['percent']
    
pd.DataFrame.from_dict(dict_density, orient='index', 
                       columns=['pixel_ta_dl','pixel_cu_dl','pixel_ac_dl','pixel_cc_dl',
                               'area_ta_dl','area_cu_dl','area_ac_dl','area_cc_dl',
                               'percent_ta_dl','percent_cu_dl','percent_ac_dl','percent_cc_dl',]).to_csv("test_density-unet.csv", index_label="file")

 14%|█▍        | 117/809 [00:32<03:16,  3.52it/s]/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
 16%|█▌        | 131/809 [00:38<03:36,  3.13it/s]/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarni

/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
 20%|█▉        | 160/809 [01:01<06:24,  1.69it/s]/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 20

/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
 24%|██▎       | 191/809 [01:24<07:07,  1.45it/s]/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 20

/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
 27%|██▋       | 222/809 [01:51<07:50,  1.25it/s]/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 20

/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
 31%|███       | 251/809 [02:17<08:21,  1.11it/s]/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 20

...

/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format h

/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format h

/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
 82%|████████▏ | 664/809 [08:15<02:19,  1.04it/s]/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 20

 86%|████████▌ | 692/809 [08:43<01:54,  1.02it/s]/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 20

/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
 93%|█████████▎| 752/809 [09:39<00:54,  1.04it/s]/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 2000 codestream with the optional JP2 file format header, which is non-conformant to the DICOM Standard (Part 5, Annex A.4.4)
  warnings.warn(
/home/jslee/miniconda3/lib/python3.9/site-packages/openjpeg/utils.py:176: UserWarning: The (7FE0,0010) Pixel Data contains a JPEG 20

In [ ]:
dict_density = {}
#for i in tqdm(range(len(df_val)),miniters=50, mininterval=5):
for i in tqdm(range(len(df_cal)),miniters=50, mininterval=5):
    file = list_cal[i]
    pred = cal_predict(model_deeplab, file, method="deeplab")
    dict_density[file] = pred['pixel'] + pred['area'] + pred['percent']
    
tmp = pd.DataFrame.from_dict(dict_density, orient='index', 
                       columns=['pixel_ta_dl','pixel_cu_dl','pixel_ac_dl','pixel_cc_dl',
                               'area_ta_dl','area_cu_dl','area_ac_dl','area_cc_dl',
                               'percent_ta_dl','percent_cu_dl','percent_ac_dl','percent_cc_dl',])
tmp.to_csv("test_density-deeplab.csv", index_label="file")

In [ ]:
name = 'test_density-deeplab_0.5-1.0.csv'
model_deeplab.load_state_dict(torch.load("model/density-deeplab_0.5-1.0.pth"))

with open(name, 'w') as result: 
    write = csv.writer(result)
    write.writerow(['file','pixel_ta_dl','pixel_cu_dl','pixel_ac_dl','pixel_cc_dl',
                               'area_ta_dl','area_cu_dl','area_ac_dl','area_cc_dl',
                               'percent_ta_dl','percent_cu_dl','percent_ac_dl','percent_cc_dl'])

for i in tqdm(range(len(df_cal)),miniters=50, mininterval=5):
    file = list_cal[i]
    pred = cal_predict(model_deeplab, file, method="deeplab")
    #dict_density[file] = pred['pixel'] + pred['area'] + pred['percent']
    with open(name, 'a') as result: 
        write = csv.writer(result)
        write.writerow([file]+pred['pixel'] + pred['area'] + pred['percent'])

In [ ]:
with open('test_density_benign.csv', 'w') as result: 
    write = csv.writer(result)
    write.writerow(['file','pixel_ta_dl','pixel_cu_dl','pixel_ac_dl','pixel_cc_dl',
                               'area_ta_dl','area_cu_dl','area_ac_dl','area_cc_dl',
                               'percent_ta_dl','percent_cu_dl','percent_ac_dl','percent_cc_dl'])

for i in tqdm(range(len(df_benign)),miniters=50, mininterval=5):
    file = list_benign[i]
    pred = cal_predict(model, file)
    with open('test_density_benign.csv', 'a') as result: 
        write = csv.writer(result)
        write.writerow([file]+pred['pixel'] + pred['area'] + pred['percent'])

In [ ]:
name = 'test_density_samsung.csv'
with open(name, 'w') as result: 
    write = csv.writer(result)
    write.writerow(['file','pixel_ta_dl','pixel_cu_dl','pixel_ac_dl','pixel_cc_dl',
                               'area_ta_dl','area_cu_dl','area_ac_dl','area_cc_dl',
                               'percent_ta_dl','percent_cu_dl','percent_ac_dl','percent_cc_dl'])
    
for i in tqdm(range(i,len(df_samsung)),miniters=50, mininterval=5):
    file = "dicom/samsung/"+df_samsung.file[i]
    try:
        pred = cal_predict(model_unet, file,method="unet")
        with open(name, 'a') as result: 
            write = csv.writer(result)
            write.writerow([file]+pred['pixel'] + pred['area'] + pred['percent'])
    except:
        pass

In [ ]:
name = 'test_density_ilsan.csv'
list_ilsan = df_ilsan.file.tolist()
with open(name, 'w') as result: 
    write = csv.writer(result)
    write.writerow(['file','pixel_ta_dl','pixel_cu_dl','pixel_ac_dl','pixel_cc_dl',
                               'area_ta_dl','area_cu_dl','area_ac_dl','area_cc_dl',
                               'percent_ta_dl','percent_cu_dl','percent_ac_dl','percent_cc_dl'])

for i in tqdm(range(len(df_ilsan)),miniters=50, mininterval=5):
    file = "dicom/ilsan/"+list_ilsan[i]
    try:
        pred = cal_predict(model_unet, file,method="unet")
        with open(name, 'a') as result: 
            write = csv.writer(result)
            write.writerow([file]+pred['pixel'] + pred['area'] + pred['percent'])
    except:
        pass

In [ ]:
name = 'test_density_cal.csv'
with open(name, 'w') as result: 
    write = csv.writer(result)
    write.writerow(['file','pixel_ta_dl','pixel_cu_dl','pixel_ac_dl','pixel_cc_dl',
                               'area_ta_dl','area_cu_dl','area_ac_dl','area_cc_dl',
                               'percent_ta_dl','percent_cu_dl','percent_ac_dl','percent_cc_dl'])

for i in tqdm(range(len(df_cal)),miniters=50, mininterval=5):
    file = list_cal[i]
    try:
        pred = cal_predict(model_unet, file,method="unet")
        with open(name, 'a') as result: 
            write = csv.writer(result)
            write.writerow([file]+pred['pixel'] + pred['area'] + pred['percent'])
    except:
        pass

In [ ]:
name = 'test_density_cal2.csv'
with open(name, 'w') as result: 
    write = csv.writer(result)
    write.writerow(['file','pixel_ta_dl','pixel_cu_dl','pixel_ac_dl','pixel_cc_dl',
                               'area_ta_dl','area_cu_dl','area_ac_dl','area_cc_dl',
                               'percent_ta_dl','percent_cu_dl','percent_ac_dl','percent_cc_dl'])

for i in tqdm(range(len(df_cal2)),miniters=50, mininterval=5):
    file = list_cal2[i]
    try:
        pred = cal_predict(model_unet, file,method="unet")
        with open(name, 'a') as result: 
            write = csv.writer(result)
            write.writerow([file]+pred['pixel'] + pred['area'] + pred['percent'])
    except:
        pass

In [ ]:
name = 'test_density_cal2_case.csv'
with open(name, 'w') as result: 
    write = csv.writer(result)
    write.writerow(['file','pixel_ta_dl','pixel_cu_dl','pixel_ac_dl','pixel_cc_dl',
                               'area_ta_dl','area_cu_dl','area_ac_dl','area_cc_dl',
                               'percent_ta_dl','percent_cu_dl','percent_ac_dl','percent_cc_dl'])

for i in tqdm(range(len(df_cal2_case)),miniters=50, mininterval=5):
    file = list_cal2_case[i]
    try:
        pred = cal_predict(model_unet, file,method="unet")
        with open(name, 'a') as result: 
            write = csv.writer(result)
            write.writerow([file]+pred['pixel'] + pred['area'] + pred['percent'])
    except:
        pass

In [ ]:
name = 'test_density_test.csv'
with open(name, 'w') as result: 
    write = csv.writer(result)
    write.writerow(['file','pixel_ta_dl','pixel_cu_dl','pixel_ac_dl','pixel_cc_dl',
                               'area_ta_dl','area_cu_dl','area_ac_dl','area_cc_dl',
                               'percent_ta_dl','percent_cu_dl','percent_ac_dl','percent_cc_dl'])

for i in tqdm(range(len(df_test)),miniters=50, mininterval=5):
    file = list_test[i]
    try:
        pred = cal_predict(model_unet, file,method="unet")
        with open(name, 'a') as result: 
            write = csv.writer(result)
            write.writerow([file]+pred['pixel'] + pred['area'] + pred['percent'])
    except:
        pass

### DenseNet

In [ ]:
dict_image = {}
for i in tqdm(range(len(df_val)),mininterval=5, miniters=50):
    file = list_val[i]
    dict_image[file] = get_risk(model_image, file)
    
df_result = pd.DataFrame.from_dict(dict_image, orient="index", columns=['softmax','sigmoid'])
df_result["softmax_prob"] = [i[1] for i in df_result.softmax]
df_result["sigmoid_prob"] = [i[1] for i in df_result.sigmoid]
df_result["softmax_label"] = [int(np.argmax(np.array(i))) for i in df_result.softmax]
df_result["sigmoid_label"] = [int(np.argmax(np.array(i))) for i in df_result.sigmoid]
df_result = df_result[['softmax_prob','sigmoid_prob','softmax_label','sigmoid_label']]

df_result.to_csv("test_image.csv",index_label = "file")

In [ ]:
with open('test_image_benign.csv', 'w') as result: 
    write = csv.writer(result)
    write.writerow(['file','softmax_prob','sigmoid_prob','softmax_label','sigmoid_label'])
    
for i in tqdm(range(len(df_benign)),mininterval=5, miniters=50):
    file = list_benign[i]
    result = get_risk(model_image, file)
    with open('test_image_benign.csv', 'a') as save: 
        write = csv.writer(save)
        write.writerow([file, result[0][1], result[1][1], np.argmax(np.array(result[0])), np.argmax(np.array(result[1]))])

In [ ]:
name = 'test_image_cal.csv'
with open(name, 'w') as result: 
    write = csv.writer(result)
    write.writerow(['file','softmax_prob','sigmoid_prob','softmax_label','sigmoid_label'])
    
for i in tqdm(range(len(df_cal)),mininterval=5, miniters=50):
    file = list_cal[i]
    result = get_risk(model_image, file)
    with open(name, 'a') as save: 
        write = csv.writer(save)
        write.writerow([file, result[0][1], result[1][1], np.argmax(np.array(result[0])), np.argmax(np.array(result[1]))])

In [ ]:
name = 'test_image_samsung.csv'
with open(name, 'w') as result: 
    write = csv.writer(result)
    write.writerow(['file','softmax_prob','sigmoid_prob','softmax_label','sigmoid_label'])
    
for i in tqdm(range(len(df_samsung)),mininterval=5, miniters=50):
    file = list_samsung[i]
    result = get_risk(model_image, file)
    with open(name, 'a') as save: 
        write = csv.writer(save)
        write.writerow([file, result[0][1], result[1][1], np.argmax(np.array(result[0])), np.argmax(np.array(result[1]))])

### Visualize

In [ ]:
# control / low density / low image risk
file = "dicom/ncc2020/25658_RCC_19000101.dcm"
print(file)
tmp = get_gradcam(model_image, file, mask_size=16, stride=32)
f.p(tmp[1])
plot_predict(model_unet, file)
plot_seg(tmp[1],tmp[4])
#plot_seg(tmp[1],tmp[2])

In [ ]:
# case / low density / high image risk
file = "dicom/ncc2020/00990_RCC_19000101.dcm"
print(file)
tmp = get_gradcam(model_image, file, mask_size=16, stride=32)
f.p(tmp[1])
plot_predict(model_unet, file)
plot_seg(tmp[1],tmp[4])
plot_seg(tmp[1],tmp[2])

In [ ]:
# control / high density / low image risk
file = "dicom/ncc2020/21284_RCC_19000101.dcm"
print(file)
tmp = get_gradcam(model_image, file, mask_size=16, stride=32)
f.p(tmp[1])
plot_predict(model_unet, file)
plot_seg(tmp[1],tmp[4])
plot_seg(tmp[1],tmp[2])

In [ ]:
file = "dicom/ncc2020/01898_LCC_19000101.dcm"
print(file)
tmp = get_gradcam(model_image, file, mask_size=16, stride=32)
f.p(tmp[1])
tmp_unet = get_predict(model_unet, file)[1]
plot_seg(tmp[1],tmp_unet, cmap="jet")
plot_seg(tmp[1],tmp_unet>=1, cmap="gray")
plot_seg(tmp[1],tmp_unet>=2, cmap="gray")
plot_seg(tmp[1],tmp_unet>=3, cmap="gray")

In [ ]:
# case / high density / high image risk
file = "dicom/ncc2020/01898_LCC_19000101.dcm"
print(file)
tmp = get_gradcam(model_image, file, mask_size=16, stride=32)
f.p(tmp[1])
plot_predict(model_unet, file)
plot_seg(tmp[1],tmp[4])
plot_seg(tmp[1],tmp[2])

In [ ]:
i=random.choice(range(len(list_dicom)))
file = list_dicom[i]
print(file, f.get_dicom(file).Manufacturer)
tmp = get_gradcam(model_image, file, mask_size=16, stride=32)
f.p(tmp[1])

In [ ]:
plot_predict(model_unet, file)

In [ ]:
i=random.choice(range(len(list_dicom)))
file = list_dicom[i]
print(file, f.get_dicom(file).Manufacturer)
tmp = get_gradcam(model_image, file, mask_size=16, stride=32)
f.p(tmp[1])

In [ ]:
plot_predict(model_unet, file)

In [ ]:
i=random.choice(range(len(list_dicom)))
file = list_dicom[i]
print(file, f.get_dicom(file).Manufacturer)
tmp = get_gradcam(model_image, file, mask_size=16, stride=32)
f.p(tmp[1])

In [ ]:
i=random.choice(range(len(list_dicom)))
file = list_dicom[i]
print(file, f.get_dicom(file).Manufacturer)
tmp = get_gradcam(model_image, file, mask_size=16, stride=32)
f.p(tmp[1])

In [ ]:
plot_predict(model_unet, file)

In [ ]:
file = "dicom/asan/37341246_LCC_20081023.dcm"
tmp = get_gradcam(model_image, file, mask_size=4, stride=8)
f.p(tmp[1])

In [ ]:
plot_predict(model_unet, file)

In [ ]:
file = "dicom/samsung/30710752_LCC_20130514.dcm"
tmp = get_gradcam(model_image, file, mask_size=8, stride=16)
f.p(tmp[1])

In [ ]:
plot_predict(model_unet, file)

In [ ]:
plot_seg(tmp[1],tmp[4])

In [ ]:
plot_seg(tmp[1],tmp[2]*-1)

In [ ]:
file = "dicom/ncc2020/23836_RCC_19000101.dcm"

In [ ]:
tmp = get_gradcam(model_image, file, mask_size=16, stride=32)
f.p(tmp[1])

In [ ]:
plot_predict(model_unet, file)

In [ ]:
plot_seg(tmp[1],tmp[4])

In [ ]:
plot_seg(tmp[1],tmp[2]*-1)

In [ ]:
file = "dicom/ilsan/0841582_LCC_20190124.dcm"

In [ ]:
tmp = get_gradcam(model_image, file, mask_size=8, stride=16)
f.p(tmp[1])

In [ ]:
plot_predict(model_unet, file)

In [ ]:
plot_seg(tmp[1],tmp[4])

In [ ]:
plot_seg(tmp[1],tmp[4])

In [ ]:
plot_seg(tmp[1],tmp[2]*-1)

In [ ]:
i=random.choice(range(len(list_dicom)))
file = list_dicom[i]
print(file, f.get_dicom(file).Manufacturer)
plot_predict(model_unet, file, method="unet")

In [ ]:
i=random.choice(range(len(list_dicom)))
file = list_dicom[i]
print(file, f.get_dicom(file).Manufacturer)
plot_predict(model_unet, file, method="unet")

In [ ]:
i=random.choice(range(len(list_dicom)))
file = list_dicom[i]
print(file, f.get_dicom(file).Manufacturer)
plot_predict(model_unet, file, method="unet")

In [ ]:
i=random.choice(range(len(list_dicom)))
file = list_dicom[i]
print(file, f.get_dicom(file).Manufacturer)
plot_predict(model_unet, file, method="unet")

In [ ]:
plot_predict(model_unet, 'dicom/ncc2018/33321785_LCC_20180125.dcm')

In [ ]:
i=random.choice(range(len(list_dicom)))
file = list_dicom[i]
print(file, f.get_dicom(file).Manufacturer)
plot_predict(model_unet, file, method="unet")

In [ ]:
i=random.choice(range(len(list_dicom)))
file = list_dicom[i]
print(file, f.get_dicom(file).Manufacturer)
plot_predict(model_unet, file, method="unet")

In [ ]:
i=random.choice(range(len(list_dicom)))
file = list_dicom[i]
print(file, f.get_dicom(file).Manufacturer)
plot_predict(model_unet, file, method="unet")

In [ ]:
plot_predict(model_deeplab, file, method="deeplab")

In [ ]:
i=random.choice(range(len(list_dicom)))
file = list_dicom[i]
print(file, f.get_dicom(file).Manufacturer)
plot_predict(model_unet, file, method="unet")

In [ ]:
plot_predict(model_deeplab, file, method="deeplab")

In [ ]:
i=random.choice(range(len(list_dicom)))
file = list_dicom[i]
print(file, f.get_dicom(file).Manufacturer)
plot_predict(model_unet, file, method="unet")

In [ ]:
plot_predict(model_deeplab, file, method="deeplab")

In [ ]:
i=random.choice(range(len(list_dicom)))
file = list_dicom[i]
print(file, f.get_dicom(file).Manufacturer)
plot_predict(model_unet, file, method="unet")

In [ ]:
plot_predict(model_deeplab, file, method="deeplab")

In [ ]:
i=random.choice(range(len(list_dicom)))
file = list_dicom[i]
print(file, f.get_dicom(file).Manufacturer)
plot_predict(model_unet, file, method="unet")

In [ ]:
plot_predict(model_deeplab, file, method="deeplab")